## Fetching data, training a classifier

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
# Add the project directory to the path
%cd /content/drive/Shared drives/CS634/CS634Project/Q2
import os, sys
sys.path.append(os.getcwd())

/content/drive/Shared drives/CS634/CS634Project/Q2


In [67]:
import sklearn
import sklearn.ensemble
import sklearn.metrics
import pandas as pd
import numpy as np

**Data Processing**

We have used unprocessed tar file of Multi-Domain Sentiment Dataset and extracted the positive and negative review file.
Having converted the xml to csv, we obtained positive.csv and negative.csv.

We used following code to merge the two file to a single file named output.csv.


*file1 = open("positive.csv", "a")*

*file2 = open("negative.csv", "r")*

*for line in file2:*

   *file1.write(line)*

*file1.close()*

*file2.close()*

and renamed the file as output.csv

**Importing the processed data and Exploratory Data Analysis**



In [68]:
  
# reading csv files
data1 = pd.read_csv('output.csv')

# Display first 5 records
data1.head()

review_text review_status
0  Bridget Jones, modern day woman, brillant and ...      positive
1  I am ordering copies for all 23 middle school ...      positive
2  As a casual piano player and a Broadway fanati...      positive
3  This is one of the best biographies I have eve...      positive
4  I read this book many, many years ago on a ver...      positive

In [69]:
# Display last 5 records
data1.tail()

review_text review_status
1995  This book provides excellent information about...      negative
1996  I really didn't enjoy this book. I am half Ita...      negative
1997  This book is a place to start at best. The mai...      negative
1998  I was barely able to finish this book. Armstro...      negative
1999  "The Burning" was a big letdown after reading ...      negative

In [70]:
#Displaying Unique classes of review_status
data1.review_status.unique()

array(['positive', 'negative'], dtype=object)

In [71]:
#Checking for null values
data1.isnull().sum()

review_text      0
review_status    0
dtype: int64

In [72]:
#review_text column is made to lower case
data1['review_text'] = data1['review_text'].str.lower()

#Assigning features as X and Label as y
X = data1['review_text']
y = data1['review_status']
print(X.shape)
print(y.shape)

(2000,)
(2000,)


In [73]:
#Importing for random.seed
import random
#Setting random seed, so values do not change for each execution.
random.seed(123)

**Splitting our dataset into train and test dataset**

In [74]:
#split our dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 125)
#test dataset size 20% , train dataset size 80%
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1600,)
(400,)
(1600,)
(400,)


**Let's use the Count vectorizer**

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, stop_words='english')
vectorizer.fit(X_train)
train_vector = vectorizer.fit_transform(X_train)
test_vector = vectorizer.transform(X_test)

Now, let's say we want to use random forests for classification. It's usually hard to understand what random forests are doing, especially with many trees.

**Random Forest Classifier model for Sentiment Analysis**

In [76]:
#Assigning rf as random forest classifier
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500, random_state = 120)
#Fiiting the model
rf.fit(train_vector, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=120,
                       verbose=0, warm_start=False)

In [77]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
#predicting Label for test data
pred = rf.predict(test_vector)
print("Random Forest Accuracy Score : " , accuracy_score(y_test.values, pred))


Random Forest Accuracy Score :  0.7575


In [78]:
sklearn.metrics.f1_score(y_test.values, pred, average='binary', pos_label= 'positive')

0.7604938271604937

We see that this classifier achieves a good F1 score. 

Now Let's try XgBoost.

**XgBoost Classifier model for Sentiment Analysis**

In [79]:
from xgboost import XGBClassifier
#Default hyperparameters
xgb_model = XGBClassifier()
xgb_model.fit(train_vector, y_train.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

Key parameters in XGBoost(the ones which would affect model quality greatly),
 * max_depth - more complex classification task, deeper the tree (default is 3)
 * n_estimators — the number of runs XGBoost will try to learn (default is 100)
 * learning_rate — learning speed (default is 0.1)





When xgb_model.fit is executed with verbose=True(default), you will see each training run evaluation quality printed out.

In [80]:

from xgboost import XGBClassifier
#Changing some hyperparameters
xgb_model = XGBClassifier(n_estimators = 500, learning_rate = 0.1,  max_depth = 5)
xgb_model.fit(train_vector, y_train.values)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [81]:
pred_xgb = xgb_model.predict(test_vector)
sklearn.metrics.f1_score(y_test.values, pred_xgb, average='binary', pos_label= 'positive')

0.7443037974683543

In [82]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print("Confusion Matrix:  \n" , confusion_matrix(y_test.values, pred_xgb))
print("\nRandom Forest Accuracy Score with XgBoost : ", accuracy_score(y_test.values, pred_xgb))
print("\nClassification Report: \n ",classification_report(y_test.values, pred_xgb))

Confusion Matrix:  
 [[152  54]
 [ 47 147]]

Random Forest Accuracy Score with XgBoost :  0.7475

Classification Report: 
                precision    recall  f1-score   support

    negative       0.76      0.74      0.75       206
    positive       0.73      0.76      0.74       194

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



# Explaining predictions using lime

In [83]:
#Installing Lime
%pip install lime

In [84]:
#Importing Lime module
import lime

In [85]:
#Construct a Pipeline from the given estimators.
from sklearn.pipeline import make_pipeline
#making pipeline using vectorizer and rf - random forest classifier
c = make_pipeline(vectorizer, rf)

Lime explainers assume that classifiers act on raw text, but sklearn classifiers act on vectorized representation of texts. For this purpose, we use sklearn's pipeline, and implement ````predict_proba```` on raw_text lists.

In [86]:
print(c.predict_proba([X_test.values[0]]))


[[0.38 0.62]]


In [87]:
#Assigning the class names as positive and negative
class_names = ['negative', 'positive']

Now we create an explainer object. We pass the ````class_names```` as an argument for prettier display.

In [88]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [89]:
y_test.values[2]

'negative'

We then generate an explanation with at most 6 features for an arbitrary document in the test set.

In [90]:
idx = 2
exp1 = explainer.explain_instance(X_test.values[idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(positive) =', c.predict_proba([X_test.values[idx]])[0, 1])
print('Probability(negative) =', c.predict_proba([X_test.values[idx]])[0, 0])
print('True class: %s' % y_test.values[idx])

Document id: 2
Probability(positive) = 0.54
Probability(negative) = 0.46
True class: negative


The classifier got this example right (it predicted positive).  
The explanation is presented below as a list of weighted features. 

In [91]:
exp1.as_list()

[('author', -0.15513518391398268),
 ('poor', -0.07703461284690943),
 ('process', 0.047542742070894435),
 ('somewhat', 0.044236682304422605),
 ('book', -0.043931443475475326),
 ('mystery', 0.031184874620601433)]

These weighted features are a linear model, which approximates the behaviour of the random forest classifier in the vicinity of the test example. Roughly, if we remove 'author' and 'poor' from the document , the prediction should move towards the opposite class (negative) by about the sum of the weights for both features. Let's see if this is the case.

In [92]:
print('Original prediction:', rf.predict_proba(test_vector[idx])[0, 1])
tmp = test_vector[idx].copy()
tmp[0, vectorizer.vocabulary_['author']] = 0
tmp[0, vectorizer.vocabulary_['poor']] = 0
print('Prediction removing some features:', rf.predict_proba(tmp)[0,1])
print('Difference:', rf.predict_proba(tmp)[0, 1] - rf.predict_proba(test_vector[idx])[0,1])

Original prediction: 0.54
Prediction removing some features: 0.728
Difference: 0.18799999999999994


The words that explain the model around this document seem very arbitrary - not much to do with either positive or negative. In fact, these are words that appear in the reviews, which make distinguishing between the classes much easier.

In [93]:
%matplotlib inline
fig = exp1.as_pyplot_figure()

In [94]:
exp1.show_in_notebook(text=False)

Finally, we can also include a visualization of the original document, with the words in the explanations highlighted. Notice how the words that affect the classifier the most are all in the review.

In [95]:
exp1.show_in_notebook(text=True)

**Summary**

As you can see in the above examples, LIME-The explainer helps the user understand what's happening behind the black-box model. With the given set of reviews (dataset), the Random Forest Classifier model was able to predict whether it is positive or negative review, and the LIME explained why. The LIME calculates the weight of the individual words locally in the given data based on the probability and classifies based on the terms that carried the highest weightage. It is evident that if the words with the highest weightage are removed, the prediction probability varies, making a difference to become the other class.

**Conclusion**

LIME is a useful tool that helps data scientists figure out why their predictive models fail and explain individual predictions. Moreover, it is easy to integrate with models built on common machine learning packages. Since LIME is model-agnostic, we will explore using it to interpret other models(Logistic Regression Classifier) in our Question 4.

This explainer works for any classifier you may want to use, as long as it implements ````predict_proba````